# Setup codes

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
from aiida.common.exceptions import NotExistent, MultipleObjectsError
from aiida.common.pluginloader import all_plugins
from aiida.orm import Code

import tempfile
import ipywidgets as ipw
from IPython.display import display, clear_output
import subprocess
import glob
import shutil

In [ ]:
computer_name = 'quantum-mobile'

In [ ]:
def get_code_templates(computer_name, tmpdir):
    print("Copying code templates from {}... ".format(computer_name), end='')
    ret = subprocess.call(["scp", computer_name+":~/.local/share/marvelnccr/*.code", tmpdir])
    print("Ok." if ret==0 else "Failed!")
    return ret==0

def is_code_installed(code_string):
    try:
        Code.get_from_string(code_string)
        return True
    except NotExistent:
        return False
    except MultipleObjectsError:
        return True

def setup_codes(computer_name):

    tmpdir = tempfile.mkdtemp()
    if not get_code_templates(computer_name, tmpdir):
        return    
       
    templates = glob.glob(tmpdir+'/*.code')
    plugins = all_plugins('calculations')
        
    for template in templates: 
        # updating code name
        with open(template,'r') as t:
            content = t.read().replace('localhost', computer_name)
        with open(template,'w') as t:
            t.write(content)

        # parsing code info
        lines =  content.split('\n')
        code_label = lines[0]
        code_string = "{}@{}".format(code_label, computer_name)
        plugin_name = lines[3]

        if is_code_installed(code_string):
            print("Code {} already installed.".format(code_string))            
            continue
        if not plugin_name in plugins:
            print("Plugin {} missing. Unable to set up code {}.".format(plugin_name, code_label))
            continue

        setup_code(template, code_label)
    shutil.rmtree(tmpdir)
        
def setup_code(template, code_label):
    print("Setting up code {}...".format(code_label), end='')
    ret = subprocess.call("cat {} | verdi code setup".format(template), shell=True)
    #! cat "{template}" | verdi code setup
    print("Ok." if ret==0 else "Failed!")
    return ret==0

In [ ]:
def on_setup_codes(b):
    with setup_code_out:
        clear_output()
        setup_codes(computer_name)

setup_code_out = ipw.Output()
btn_setup_codes = ipw.Button(description="Setup Codes")
btn_setup_codes.on_click(on_setup_codes)
display(btn_setup_codes, setup_code_out)

In [ ]:
# DEPRECATED
# def setup_codes_old(computer_name):
#     computer = Computer.get(computer_name)
    
#     qe_path = "/home/ubuntu/codes/q-e-qe-6.2.1-fda42b8045f604ef8cc8400dac1b2a06cb3f6e91/bin/"
#     cp2k_path = "/home/ubuntu/codes/cp2k-5.1/exe/"
    
#     # CP2K
#     cp2k_version = '6.0'
#     cp2k_revision = '18265'
#     code_label = "cp2k_{}_{}".format(cp2k_version, cp2k_revision)
#     code_full_name = "{}@{}".format(code_label, computer_name)
#     try:
#         Code.get_from_string(code_full_name)
#     except (NotExistent, MultipleObjectsError):            
#         code = Code(remote_computer_exec=(computer, cp2k_path + "cp2k.ssmp"))
#         code.label = code_label
#         code.description = "CP2K Code"
#         code.set_input_plugin_name("cp2k")
#         prepend_text  = "### code prepend_text start ###\n"
#         prepend_text += "module load CP2K\n"
#         prepend_text += "export CP2K_DATA_DIR=/home/ubuntu/codes/cp2k-5.1/\n"
#         prepend_text += "### code prepend_text end ###\n"
#         code.set_prepend_text(prepend_text)
#         code.set_append_text("")
#         code.store()
#         code._reveal()
#         ! verdi code show "{code_full_name}"
#     else:
#         print("Code '{}' already existent".format(code_full_name))        
    
#     # QuantumESPRESSO pw
#     for codename in ['pw', 'pp', 'ph', 'projwfc', 'matdyn', 'q2r']:
#         code_label = "{}-{}".format(codename, qe_version)
#         code_full_name = "{}@{}".format(code_label, computer_name)
#         try:
#             Code.get_from_string(code_full_name)
#         except (NotExistent, MultipleObjectsError):            
#             code = Code(remote_computer_exec=(computer, qe_path + "{}.x".format(codename)))
#             code.label = code_label
#             code.description = "Quantum ESPRESSO {}.x version {}".format(codename, qe_version)
#             code.set_input_plugin_name("quantumespresso.{}".format(codename))
#             prepend_text  = "### code prepend_text start ###\n"
#             prepend_text += "module load QuantumESPRESSO/6.1.0-CrayIntel-17.08\n"
#             prepend_text += "### code prepend_text end ###\n"
#             code.set_prepend_text(prepend_text)
#             code.set_append_text("")
#             code.store()
#             code._reveal()
#             ! verdi code show "{code_full_name}"
#         else:
#             print("Code '{}' already existent".format(code_full_name))